In [ ]:
import pandas as pd

EXCEL_FILE = "data/Muesli Project raw data 21-3.xlsx"

def import_xls(file, sheet, header=0):
    df = pd.read_excel(file, sheet_name = sheet, header=header)
    return df

    

In [ ]:
df_orders = import_xls(EXCEL_FILE, "Orders", header=1)
df_order_process = import_xls(EXCEL_FILE, "Order Process Data")
df_intern = import_xls(EXCEL_FILE, "InternData Study")
df_campaign = import_xls(EXCEL_FILE, "Campaign Data")
df_list = [df_orders, df_order_process, df_intern, df_campaign]


In [ ]:
def column_rename(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')
    return df

In [ ]:
for df in df_list:
    column_rename(df)
    #display(df)


In [ ]:
# for df in df_list:
#     display(df.info())
#     display(df.describe())

In [ ]:
df_list = [df_orders, df_order_process, df_intern, df_campaign]

df_all = pd.merge(df_orders, df_order_process, how='left', on='order_id')
df_all = pd.merge(df_all, df_intern, how='left', on='order_id')
df_all = pd.merge(df_all, df_campaign, how='left', on='order_id')

df = pd.merge(df_orders, df_order_process, how='inner', on='order_id') 

In [ ]:
display(df_all.head())
display(df.head())

display(df_all.info())
display(df.info()) 

In [ ]:
df["order_id"].nunique()
drop_list = ['customer_name', 'country/region', 'city', 'state', 'postal_code', 'region'] 
df = df.drop(drop_list, axis=1)


In [ ]:
display(df.head(10))


In [ ]:
df[df["order_date_x"] != df["order_date_y"]]
df[df["order_date_x"] == df["order_date_y"]].count()



In [ ]:
df.drop("order_date_y", axis=1, inplace =True)


In [ ]:
df.rename(columns = {"order_date_x": "order_date"}, inplace=True)
display(df)

In [ ]:
df["pretransportation_duration"] = df["on_truck_scan_date"] - df["order_date"]
#df["pretransportation_duration"] = df["pretransportation_duration"].astype("int")
display(df)

In [ ]:
df.describe()